In [2]:
import urllib2, json, pickle
import numpy as np
import pandas as pd

In [74]:
q = 'https://api.istex.fr/document/?q=((categories.wos:%22Industrial%20Relations%22)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id'
f = 5000
q+'&from='+str(f)

'https://api.istex.fr/document/?q=((categories.wos:%22Industrial%20Relations%22)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id&from=5000'

In [40]:
for i in range(data['total'] / 5000 +1):
    print i

0
1


In [162]:
def query_from(q, f):
    q = q+'&from='+str(f)
    response = urllib2.urlopen(q)
    data = json.load(response)
    subject_ids = np.array(range(len(data['hits'])), dtype=np.object)
    for (i, hit) in enumerate(data['hits']):
        subject_ids[i] = hit['id']
    return subject_ids

def query(q):
    response = urllib2.urlopen(q)
    data = json.load(response)
    nb_requests = 1 + data['total'] / 1000
    if nb_requests > 10: # maximum number of pages due to API pagination restrection
        nb_requests = 10
    subject_ids = query_from(q, 0)
    for i in range(nb_requests)[1:]:
        f = i * 1000
        next_request = query_from(q, f)
        subject_ids = np.hstack((subject_ids, next_request))
    return subject_ids.tolist()


In [157]:
q = 'https://api.istex.fr/document/?q=((categories.wos:%22Industrial%20Relations%22)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id'
f = 6000
len(query_from(q, f))

625

In [52]:
def find_intersection(list_a, list_b):
    return list(set(list_a) & set(list_b))
a = ['hussein', 'tayseer', 'al-natsheh']
b = ['loay', 'hussein', 'al-natsheh']
find_intersection(a, b)

['al-natsheh', 'hussein']

In [6]:
inv_index = json.load(open('../RecSys_Exp_files/182_381_vec150_results/output_paragraph_inversed_index.json','rb'))

In [7]:
print 'original inversed_index'
print inv_index.items()[:5]
inversed_index = dict()
for (k, v) in inv_index.items():
    key = k.split('_')[1]
    inversed_index[key] = v
print 'processed inversed_index'
print inversed_index.items()[:5]

original inversed_index
[(u'ISTEX_D89FA3AC3521074D46F4245762153DF497BFFA1F', 2002320), (u'ISTEX_18EAF4D6A126B077EB38667801D1B7292F32FF49', 2483732), (u'ISTEX_5F91044435FCC4FABB9F02E31467DCFE75F4A7BE', 1429049), (u'ISTEX_EBB03C272511D346B3D68CA10EBD929CE7A31DB8', 128349), (u'ISTEX_5776E10BEF3FF14794D60A604B2CA12B1695894B', 1225023)]
processed inversed_index
[(u'FCF1393F9B8136AC08FB67E88F94F3CF62C17288', 3517138), (u'482E1102A1114327A744FD2ADB4D9F8FF7E9A70B', 751643), (u'A81022B6295AE66F68A10222C3B94A06B033C1BA', 3983232), (u'F27F00C86A2994208F0143A5CD2217AB87B21A64', 1906438), (u'939B5153F5A4B8053FFAF8E7BCE062B341DB9BEB', 3603049)]


In [171]:
def stat_topic(topic):
    _all = query('https://api.istex.fr/document/?q=((title:%22'+topic+'%22%20OR%20subject.value:%22'+topic+'%22%20OR%20keywords:%22'+topic+'%22%20OR%20abstract:%22'+topic+'%22)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id')
    print 'all:', len(_all)
    _abst_title = query('https://api.istex.fr/document/?q=((title:%22'+topic+'%22%20OR%20abstract:%22'+topic+'%22)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id')
    print 'size of abstract_title:',len(_abst_title)
    print 'abstract_title that exists in SVD(possible training set)', len(find_intersection(_abst_title,inversed_index.keys()))
    _key_subj = query('https://api.istex.fr/document/?q=((subject.value:%22'+topic+'%22%20OR%20keywords:%22'+topic+'%22%20)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id')
    print 'size of key_subject:', len(_key_subj)
    at_all_inter = find_intersection(_all, _abst_title)
    print 'intersection all and abst_title (training set):', len(at_all_inter)
    ks_at_inter = find_intersection(_key_subj, _abst_title)
    print 'intersection key_sub and abst_title:', len(ks_at_inter)
    ks_all_inter = find_intersection(_key_subj,_all)
    print 'intersection key_sub and all:', len(ks_all_inter)
    _inter = {x for x in _key_subj if x not in _abst_title}
    print 'size of key_sub but not abst_tile:', len(_inter)
    print 'key_sub but not abst_tile that exists in SVD(possible test set)', len(find_intersection(_inter,inversed_index.keys()))
    _categories = query('https://api.istex.fr/document/?q=((categories.wos:%22'+topic+'%22%20AND%20corpusName:%22elsevier%22)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id')
    if len(_categories) == 0:
        _categories = query('https://api.istex.fr/document/?q=((categories.wos:%22'+topic+'%22)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id')
    print 'len of category='+topic+':', len(_categories)
    _test = {x for x in _categories if x not in _all}
    print 'test length considering categories:', len(_test)
    _initialcorpus = find_intersection(_all, inversed_index.keys())
    print 'size of initial corpus that exists in SVD:', len(_initialcorpus)
    interaction_test = find_intersection(_test, inversed_index.keys())
    print 'size of test set that exists in SVD:', len(interaction_test)

In [93]:
stat_topic('genetic%20toxicology')

all: 213
abstract_title (training set): 160
key_subject: 75
intersection all and abst_title (training set): 160
intersection key_sub and abst_title: 22
intersection key_sub and all: 75
key_sub but not abst_tile (test set): 53
len of category=genetic%20toxicology: 0
new test length considering categories: 0
size of initial corpus that exists in SVD: 198
size of test set that exists in SVD: 0


In [96]:
stat_topic('Computational%20Biology')

all: 431
abstract_title (training set): 144
key_subject: 296
intersection all and abst_title (training set): 144
intersection key_sub and abst_title: 9
intersection key_sub and all: 296
key_sub but not abst_tile (test set): 287
len of category=Computational%20Biology: 5382
new test length considering categories: 5380
size of initial corpus that exists in SVD: 394
size of test set that exists in SVD: 5099


In [90]:
stat_topic('Veterinary%20Science')# small testset

all: 41
abstract_title (training set): 37
key_subject: 4
intersection all and abst_title (training set): 37
intersection key_sub and abst_title: 0
intersection key_sub and all: 4
key_sub but not abst_tile (test set): 4
len of category=Veterinary%20Science: 0
new test length considering categories: 0
size of initial corpus that exists in SVD: 36
size of test set that exists in SVD: 0


In [97]:
stat_topic('Biodiversity%20Conservation')

all: 600
abstract_title (training set): 500
key_subject: 147
intersection all and abst_title (training set): 500
intersection key_sub and abst_title: 47
intersection key_sub and all: 147
key_sub but not abst_tile (test set): 100
len of category=Biodiversity%20Conservation: 1554
new test length considering categories: 1542
size of initial corpus that exists in SVD: 399
size of test set that exists in SVD: 1468


In [98]:
stat_topic('Critical%20Care%20Medicine')

all: 115
abstract_title (training set): 109
key_subject: 14
intersection all and abst_title (training set): 109
intersection key_sub and abst_title: 8
intersection key_sub and all: 14
key_sub but not abst_tile (test set): 6
len of category=Critical%20Care%20Medicine: 3396
new test length considering categories: 3383
size of initial corpus that exists in SVD: 89
size of test set that exists in SVD: 3138


In [99]:
stat_topic('Industrial%20Relations')

all: 1546
abstract_title (training set): 1347
key_subject: 355
intersection all and abst_title (training set): 1347
intersection key_sub and abst_title: 156
intersection key_sub and all: 355
key_sub but not abst_tile (test set): 199
len of category=Industrial%20Relations: 6625
new test length considering categories: 5817
size of initial corpus that exists in SVD: 949
size of test set that exists in SVD: 3877


In [101]:
stat_topic('sign%20language')

all: 296
abstract_title (training set): 272
key_subject: 100
intersection all and abst_title (training set): 272
intersection key_sub and abst_title: 76
intersection key_sub and all: 100
key_sub but not abst_tile (test set): 24
len of category=sign%20language: 0
new test length considering categories: 0
size of initial corpus that exists in SVD: 226
size of test set that exists in SVD: 0


In [100]:
stat_topic('systematic%20innovation')#too small test set

all: 6
abstract_title (training set): 5
key_subject: 1
intersection all and abst_title (training set): 5
intersection key_sub and abst_title: 0
intersection key_sub and all: 1
key_sub but not abst_tile (test set): 1
len of category=systematic%20innovation: 0
new test length considering categories: 0
size of initial corpus that exists in SVD: 3
size of test set that exists in SVD: 0


In [47]:
stat_topic('science%20fiction')

all: 194
abstract_title (training set): 182
key_subject: 33
intersection all and abst_title (training set): 182
intersection key_sub and abst_title: 21
intersection key_sub and all: 33
key_sub but not abst_tile (test set): 12
len of category=science%20fiction: 0
new test length considering categories: 0


In [30]:
stat_topic('natural%20language%20understanding')# small test

all: 69
abstract_title (training set): 60
key_subject: 14
intersection all and abst_title (training set): 60
intersection key_sub and abst_title: 5
intersection key_sub and all: 14
key_sub but not abst_tile (test set): 9


In [31]:
stat_topic('deep%20learning')# small test

all: 77
abstract_title (training set): 71
key_subject: 14
intersection all and abst_title (training set): 71
intersection key_sub and abst_title: 8
intersection key_sub and all: 14
key_sub but not abst_tile (test set): 6


In [102]:
stat_topic('complex%20system')

all: 1232
abstract_title (training set): 1194
key_subject: 42
intersection all and abst_title (training set): 1194
intersection key_sub and abst_title: 4
intersection key_sub and all: 42
key_sub but not abst_tile (test set): 38
len of category=complex%20system: 0
new test length considering categories: 0
size of initial corpus that exists in SVD: 1027
size of test set that exists in SVD: 0


In [103]:
stat_topic('complex%20network')

all: 681
abstract_title (training set): 667
key_subject: 22
intersection all and abst_title (training set): 667
intersection key_sub and abst_title: 8
intersection key_sub and all: 22
key_sub but not abst_tile (test set): 14
len of category=complex%20network: 0
new test length considering categories: 0
size of initial corpus that exists in SVD: 571
size of test set that exists in SVD: 0


In [104]:
stat_topic('Agricultural%20Economics')

all: 149
abstract_title (training set): 138
key_subject: 15
intersection all and abst_title (training set): 138
intersection key_sub and abst_title: 4
intersection key_sub and all: 15
key_sub but not abst_tile (test set): 11
len of category=Agricultural%20Economics: 939
new test length considering categories: 932
size of initial corpus that exists in SVD: 138
size of test set that exists in SVD: 911


In [105]:
stat_topic('Speech-Language%20Pathology')

all: 68
abstract_title (training set): 61
key_subject: 12
intersection all and abst_title (training set): 61
intersection key_sub and abst_title: 5
intersection key_sub and all: 12
key_sub but not abst_tile (test set): 7
len of category=Speech-Language%20Pathology: 3705
new test length considering categories: 3696
size of initial corpus that exists in SVD: 32
size of test set that exists in SVD: 3611


In [144]:
stat_topic('Developmental%20Biology')

all: 840
size of abstract_title: 650
abstract_title that exists in SVD(possible training set) 570
size of key_subject: 216
intersection all and abst_title (training set): 650
intersection key_sub and abst_title: 26
intersection key_sub and all: 216
size of key_sub but not abst_tile: 190
key_sub but not abst_tile that exists in SVD(possible test set) 182
len of category=Developmental%20Biology: 10000
test length considering categories: 9941
size of initial corpus that exists in SVD: 752
size of test set that exists in SVD: 9304


In [145]:
stat_topic('Biology')

all: 10000
size of abstract_title: 10000
abstract_title that exists in SVD(possible training set) 8560
size of key_subject: 9675
intersection all and abst_title (training set): 6708
intersection key_sub and abst_title: 783
intersection key_sub and all: 4075
size of key_sub but not abst_tile: 8892
key_sub but not abst_tile that exists in SVD(possible test set) 8147
len of category=Biology: 10000
test length considering categories: 9848
size of initial corpus that exists in SVD: 8628
size of test set that exists in SVD: 7788


In [158]:
stat_topic('Infectious%20Diseases')

all: 4465
size of abstract_title: 2981
abstract_title that exists in SVD(possible training set) 2588
size of key_subject: 1626
intersection all and abst_title (training set): 2981
intersection key_sub and abst_title: 142
intersection key_sub and all: 1626
size of key_sub but not abst_tile: 1484
key_sub but not abst_tile that exists in SVD(possible test set) 1375
len of category=Infectious%20Diseases: 10000
test length considering categories: 9865
size of initial corpus that exists in SVD: 3963
size of test set that exists in SVD: 8375


In [172]:
stat_topic('Peripheral%20Vascular%20Disease')

all: 1182
size of abstract_title: 992
abstract_title that exists in SVD(possible training set) 863
size of key_subject: 299
intersection all and abst_title (training set): 992
intersection key_sub and abst_title: 109
intersection key_sub and all: 299
size of key_sub but not abst_tile: 190
key_sub but not abst_tile that exists in SVD(possible test set) 164
len of category=Peripheral%20Vascular%20Disease: 10000
test length considering categories: 9886
size of initial corpus that exists in SVD: 1027
size of test set that exists in SVD: 9686


In [166]:
stat_topic('medicine')

all: 10000
size of abstract_title: 10000
abstract_title that exists in SVD(possible training set) 8516
size of key_subject: 10000
intersection all and abst_title (training set): 7718
intersection key_sub and abst_title: 1698
intersection key_sub and all: 3980
size of key_sub but not abst_tile: 8302
key_sub but not abst_tile that exists in SVD(possible test set) 7254
len of category=medicine: 10000
test length considering categories: 9839
size of initial corpus that exists in SVD: 8434
size of test set that exists in SVD: 9194


In [170]:
stat_topic('disease')

all: 10000
size of abstract_title: 10000
abstract_title that exists in SVD(possible training set) 8648
size of key_subject: 10000
intersection all and abst_title (training set): 4460
intersection key_sub and abst_title: 1346
intersection key_sub and all: 3480
size of key_sub but not abst_tile: 8654
key_sub but not abst_tile that exists in SVD(possible test set) 7989
len of category=disease: 10000
test length considering categories: 9873
size of initial corpus that exists in SVD: 8757
size of test set that exists in SVD: 9675


In [173]:
stat_topic('Peripheral')

all: 10000
size of abstract_title: 10000
abstract_title that exists in SVD(possible training set) 9011
size of key_subject: 7678
intersection all and abst_title (training set): 8159
intersection key_sub and abst_title: 2178
intersection key_sub and all: 4017
size of key_sub but not abst_tile: 5500
key_sub but not abst_tile that exists in SVD(possible test set) 5077
len of category=Peripheral: 10000
test length considering categories: 9859
size of initial corpus that exists in SVD: 9021
size of test set that exists in SVD: 9657


In [174]:
stat_topic('Vascular')

all: 10000
size of abstract_title: 10000
abstract_title that exists in SVD(possible training set) 9038
size of key_subject: 9483
intersection all and abst_title (training set): 7821
intersection key_sub and abst_title: 3117
intersection key_sub and all: 5295
size of key_sub but not abst_tile: 6366
key_sub but not abst_tile that exists in SVD(possible test set) 5813
len of category=Vascular: 10000
test length considering categories: 9432
size of initial corpus that exists in SVD: 9003
size of test set that exists in SVD: 9241


In [175]:
stat_topic('blood')

all: 10000
size of abstract_title: 10000
abstract_title that exists in SVD(possible training set) 8809
size of key_subject: 10000
intersection all and abst_title (training set): 5614
intersection key_sub and abst_title: 2493
intersection key_sub and all: 5010
size of key_sub but not abst_tile: 7507
key_sub but not abst_tile that exists in SVD(possible test set) 6640
len of category=blood: 0
test length considering categories: 0
size of initial corpus that exists in SVD: 8823
size of test set that exists in SVD: 0
